# Installations

In [ ]:
# !pip install  "G:/16860/DM_BIGDATA/DM_Digital_y_CJ/Utilidades/Python/Packages/utm-0.5.0.tar.gz"

In [ ]:
# !pip install "branca-0.4.0.tar.gz"

In [ ]:
# !pip install "folium-0.10.1.tar.gz"

# Import Packages

In [ ]:
import folium
from folium import plugins
from folium.plugins import TimeSliderChoropleth
import pandas as pd
import json
import os
from zipfile import ZipFile
from datetime import timedelta
import plotly.express as px
import plotly.offline

# Import Data

## Import Geo

In [ ]:
with open('./data/comunidades-autonomas-espanolas.json') as f:
  ccaa = json.load(f)

In [ ]:
ccaa_clean = {'type': 'FeatureCollection'
             ,'features':[{'type':'Feature'
                  ,'id':ccaa[i]['fields']['codigo']
                  ,'properties':{'name':ccaa[i]['fields']['texto']}
                 , 'geometry':{'type':ccaa[i]['fields']['geo_shape']['type']
                              , 'coordinates':ccaa[i]['fields']['geo_shape']['coordinates']}}
            for i in range(0,19)]
             }

In [ ]:
# for i in range(0,19):
#     print(ccaa_clean['features']

In [ ]:
# ccaa_clean['features'][0]

## Import Contagion Data

In [ ]:
# virus = pd.read_excel('./data/COVID_19_SPAIN_OFFICIAL_DATA.xlsx')

In [ ]:
# https://github.com/datadista/datasets/tree/master/COVID%2019

In [ ]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('./data/datasets-master.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('data')

In [ ]:
casos = pd.read_csv('./data/datasets-master/COVID 19/ccaa_covid19_casos_long.csv'
                    ,dtype={'cod_ine':'str'},parse_dates=True)
altas = pd.read_csv('./data/datasets-master/COVID 19/ccaa_covid19_altas_long.csv'
                    ,dtype={'cod_ine':'str'},parse_dates=True)
fallecidos = pd.read_csv('./data/datasets-master/COVID 19/ccaa_covid19_fallecidos_long.csv'
                         ,dtype={'cod_ine':'str'},parse_dates=True)
hosp = pd.read_csv('./data/datasets-master/COVID 19/ccaa_covid19_hospitalizados_long.csv'
                   ,dtype={'cod_ine':'str'},parse_dates=True)
uci = pd.read_csv('./data/datasets-master/COVID 19/ccaa_covid19_uci_long.csv'
                  ,dtype={'cod_ine':'str'},parse_dates=True)

In [ ]:
casos.rename(columns={'total':'casos'},inplace=True)
altas.rename(columns={'total':'altas'},inplace=True)
fallecidos.rename(columns={'total':'fallecidos'},inplace=True)
hosp.rename(columns={'total':'hosp'},inplace=True)
uci.rename(columns={'total':'uci'},inplace=True)

In [ ]:
# max(casos['fecha'])

### merge partials

In [ ]:
virus = casos.merge(fallecidos,how='outer',on=['fecha','cod_ine','CCAA'])
virus = virus.merge(altas,how='outer',on=['fecha','cod_ine','CCAA'])
virus = virus.merge(hosp,how='outer',on=['fecha','cod_ine','CCAA'])
virus = virus.merge(uci,how='outer',on=['fecha','cod_ine','CCAA'])
virus.fillna(0,inplace=True)
# virus.head()

In [ ]:
virus.drop(virus[virus['CCAA']=='Total'].index, axis=0,inplace=True)
# virus.tail()

### add manually latest day

In [ ]:
# virus_new = pd.read_excel('./data/CCAA_nuevos.xlsx',dtype={'cod_ine':'str'},parse_dates=True)
# virus_new.drop(columns=['casos/pop','nuevos'],inplace=True)
# virus = virus_new.append(virus, ignore_index=True,sort=True)

In [ ]:
# virus.sort_values(['fecha','CCAA']).tail(25)

### adjust reporting date

In [ ]:
# adjust date of reporting
virus['fecha'] = pd.to_datetime(virus['fecha'])
virus['fecha'] = virus['fecha'] - timedelta(days=1)
virus['fecha'] = pd.to_datetime(virus['fecha'])
# max(virus['fecha'])

In [ ]:
# virus.sort_values(ascending=False, by=['fecha','casos']).head()

In [ ]:
# virus.groupby('fecha')[['casos','fallecidos','altas']].sum().sort_values(ascending=False,by='fecha')

### add population

In [ ]:
# https://www.ine.es/jaxiT3/Tabla.htm?t=2853&L=0
pop = {
'Andalucía': 8414240,
'Aragón' : 1319291,
'Asturias': 1022800,
'Baleares': 1149460,
'Canarias': 2153389,
'Cantabria': 581078,
'Castilla-La Mancha': 2032863,
'Castilla y León': 2399548,
'Cataluña': 7675217,
'Ceuta': 84777,
'C. Valenciana': 5003769,
'Extremadura': 1067710,
'Galicia': 2699499,
'Madrid' : 6663394,
'Melilla': 86487,
'Murcia': 1493898,
'Navarra': 654214,
'País Vasco': 2207776,
'La Rioja': 316798
}
virus['pop'] = virus['CCAA'].map(pop)
# virus[virus['fecha']==max(virus['fecha'])]

### add UCI data

In [ ]:
uci_disp = pd.read_csv('./data/datasets-master/COVID 19/ccaa_camas_uci_2017.csv'
                       ,dtype={'cod_ine':'str'}).drop(columns=['Públicos','Privados'])
uci_disp.rename(columns={'Total':'uci_disp'},inplace=True)
# uci_disp

In [ ]:
# http://www.datadista.com
# https://datadista.com/coronavirus/camas-uci/
# update marzo de 2020
uci_disp_upd = pd.DataFrame({'cod_ine':['17','18','19']
                              ,'CCAA':['La Rioja','Ceuta','Melilla']
                              ,'uci_disp':[22,8,7]})
uci_disp = uci_disp.append(uci_disp_upd,ignore_index=True,sort=True)
uci_disp.drop(uci_disp[uci_disp['cod_ine']=='21'].index, axis=0, inplace=True)
# uci_disp

In [ ]:
virus = virus.merge(uci_disp[['cod_ine','uci_disp']],how='left',on='cod_ine')

### compute ratios

In [ ]:
virus['casos/pop'] = virus['casos']/virus['pop']*100000
virus['uci/uci_disp'] = virus['uci']/virus['uci_disp']
virus = virus[['fecha', 'cod_ine', 'CCAA', 'casos', 'fallecidos','altas','hosp','uci'
               ,'uci_disp','pop','casos/pop','uci/uci_disp']]

## Latest data

In [ ]:
virus_latest = virus[virus['fecha']==max(virus['fecha'])]
# virus_latest

# Plot maps

In [ ]:
if not os.path.exists('images'):
    os.makedirs('images')

### folium 0

In [ ]:
# m = folium.Map([40, -2], zoom_start=5)
# m

In [ ]:
# m = folium.Map([40, -2], zoom_start=5)
# # mark each station as a point
# for index, row in virus.iterrows():
#     folium.CircleMarker([row['lat'], row['lon']],
#                         radius=5,
#                         popup=row.index,
#                         fill_color="#3db7e4", # divvy color
#                        ).add_to(m)
# m

In [ ]:
# m = folium.Map([40, -2], zoom_start=5)
# for i in range(0, len(virus_latest)):
#     folium.Circle(
#         location=[virus_latest.iloc[i]['lat'], virus_latest.iloc[i]['lon']],
#         color='orange', 
#         tooltip =   '<li><bold>Comunidad : '+str(virus_latest.iloc[i]['CCAA'])+
#                     '<li><bold>Contagiados : '+str(virus_latest.iloc[i]['TOTAL_INFECTED'])+
#                     '<li><bold>Muertos : '+str(virus_latest.iloc[i]['TOTAL_DEATHS']),
#         radius=int(virus_latest.iloc[i]['TOTAL_INFECTED'])*20,
#         fill_color='orange').add_to(m)
# m

In [ ]:
# m = folium.Map([40, -2], zoom_start=5)
# for i in range(0,len(virus_def)):
#    folium.Circle(
#       location=[virus_def.iloc[i]['lat'], virus_def.iloc[i]['lon']],
# #       popup=virus_def.iloc[i]['Comunidades'],
#       radius=int(virus_def.iloc[i]['TOTAL_DEATHS'])*100,
#       color='crimson',
#       fill=True,
#       fill_color='crimson',
#       tooltip =   '<li><bold>Comunidad : '+str(virus_def.iloc[i]['CCAA'])+
#       '<li><bold>Contagiados : '+str(virus_def.iloc[i]['TOTAL_INFECTED'])+
#       '<li><bold>Muertos : '+str(virus_def.iloc[i]['TOTAL_DEATHS']),
#    ).add_to(m)
# m

In [ ]:
# virus['TOTAL_INFECTED'].sum()

### foliun 1

In [ ]:
# virus_latest.head(1)

In [ ]:
m = folium.Map([40, -2], zoom_start=6)

# bins = [i for i in range(0,15000,1000)]

folium.Choropleth(
    geo_data=ccaa_clean,
    name='choropleth',
    data=virus_latest,
    columns=['cod_ine', 'casos'],
    key_on='feature.id',
    fill_color='OrRd',
#     bins=[float(x) for x in bins],
#     reset=True,
    fill_opacity=0.8,
    line_opacity=0.5,
    legend_name='Contagios'
).add_to(m)
m.save('./images/CCAA_folium_contagios_latest.html')
# m

In [ ]:
# virus_latest.head(1)

In [ ]:
m = folium.Map([40, -2], zoom_start=6)

# bins = [i for i in range(0,15000,1000)]
# for i in range(0,len(virus_latest)):
folium.Choropleth(
    geo_data=ccaa_clean,

    name='choropleth',
    data=virus_latest,
    columns=['cod_ine', 'casos/pop'],
    key_on='feature.id',
    fill_color='OrRd',
#     bins=[float(x) for x in bins],
#     reset=True,
    fill_opacity=0.8,
    line_opacity=0.5,
    legend_name='Contagios per 100.000 habitantes',
#         tooltip =  '<li><bold>Comunidad : '+str(virus_latest.iloc[i]['CCAA'])
#       '<li><bold>Contagiados : '+str(virus_def.iloc[i]['TOTAL_INFECTED'])+
#       '<li><bold>Muertos : '+str(virus_def.iloc[i]['TOTAL_DEATHS']),
).add_to(m)
# folium.LayerControl().add_to(m)
m.save('./images/CCAA_folium_contagios_pop_latest.html')
# m

## plotly express

## Latest Scenario for CCAA

### Confirmed cases

In [ ]:
fig = px.choropleth(virus_latest, geojson=ccaa_clean, color="casos",
                    locations="cod_ine", featureidkey="id",
                    projection="mercator",
                    color_continuous_scale="OrRd", 
#                     title='Contagios por CCAA',
                    labels={'casos':'Contagios'}
                   )
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image('./images/CCAA_contagios_latest.png')
plotly.offline.plot(fig, filename='./images/CCAA_contagios_latest.html',auto_open=False)
fig.show()

### Confirmed cases every 100.000 people

In [ ]:
fig = px.choropleth(virus_latest, geojson=ccaa_clean, color="casos/pop",
                    locations="cod_ine", featureidkey="id",
                    projection="mercator",
                    color_continuous_scale="OrRd", 
#                     title='Contagios por CCAA',
                    labels={'casos/pop':'Contagios por 100.000 habitantes'}
                   )
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image('./images/CCAA_contagios_pop_latest.png')
plotly.offline.plot(fig, filename='./images/CCAA_contagios_pop_latest.html',auto_open=False)
fig.show()

### Confirmed cases treated in Intensive Care Unit (ICU) over number of ICU available in 2017

In [ ]:
fig = px.choropleth(virus_latest, geojson=ccaa_clean, color="uci/uci_disp",
                    locations="cod_ine", featureidkey="id",
                    projection="mercator",
                    color_continuous_scale="OrRd", 
#                     title='Contagios por CCAA',
                    labels={'uci/uci_disp':'Personas en UCI sb. UCI disponibles (2017)'}
                   )
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image('./images/CCAA_uci_uci_disp_latest.png')
plotly.offline.plot(fig, filename='./images/CCAA_uci_uci_disp_latest.html',auto_open=False)
# fig.show()

## Evolution for CCAA

### Evolution over time of confirmed cases

In [ ]:
virus_temp = virus
virus_temp['fecha'] = pd.to_datetime(virus_temp['fecha'])
virus_temp['fecha'] = virus_temp['fecha'].dt.strftime('%m/%d/%Y')
virus_temp.sort_values(by='fecha',inplace=True)
fig = px.choropleth(virus_temp, geojson=ccaa_clean, color="casos",
                    locations="cod_ine", featureidkey="id",
                    projection="mercator",
                    color_continuous_scale="OrRd", 
#                     title='Contagios por CCAA',
                    animation_frame='fecha',
                    range_color= [0, max(virus_temp['casos'])]
                    , labels={'casos':'Contagios','fecha':'Fecha'})
                   
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plotly.offline.plot(fig, filename='./images/CCAA_contagios_evol.html',auto_open=False)
# fig.show()

### Evolution over time of confirmed cases every 100.000 people

In [ ]:
virus_temp = virus
virus_temp['fecha'] = pd.to_datetime(virus_temp['fecha'])
virus_temp['fecha'] = virus_temp['fecha'].dt.strftime('%m/%d/%Y')
virus_temp.sort_values(by='fecha',inplace=True)
fig = px.choropleth(virus_temp, geojson=ccaa_clean, color="casos/pop",
                    locations="cod_ine", featureidkey="id",
                    projection="mercator",
                    color_continuous_scale="OrRd", 
                    title='Contagios por CCAA',
                    animation_frame='fecha',
                    range_color= [0, max(virus_temp['casos/pop'])]
                    , labels={'casos/pop':'Contagios por 100.000 habitantes','fecha':'Fecha'},)
                   
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plotly.offline.plot(fig, filename='./images/CCAA_contagios_pop_evol.html',auto_open=False)
# fig.show()

### Evolution over time of confirmed cases treated in Intensive Care Unit (ICU) over number of ICU available in 2017

In [ ]:
virus_temp = virus
virus_temp['fecha'] = pd.to_datetime(virus_temp['fecha'])
virus_temp['fecha'] = virus_temp['fecha'].dt.strftime('%m/%d/%Y')
virus_temp.sort_values(by='fecha',inplace=True)
fig = px.choropleth(virus_temp, geojson=ccaa_clean, color="uci/uci_disp",
                    locations="cod_ine", featureidkey="id",
                    projection="mercator",
                    color_continuous_scale="OrRd", 
                    title='Contagios por CCAA',
                    animation_frame='fecha',
                    range_color= [0, max(virus_temp['uci/uci_disp'])]
                    , labels={'uci/uci_disp':'Personas en UCI sb. UCI disponibles (2017)','fecha':'Fecha'},)
                   
                   
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plotly.offline.plot(fig, filename='./images/CCAA_uci_disp_evol.html',auto_open=False)
fig.show()